In [13]:
import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
import re
import random
import json
# 高阶智能驾驶汽车品牌、型号、自动驾驶系统及硬件配置
# 从文件加载 JSON 数据
with open('car_brands_models_systems.json', 'r') as file:
    car_brands_models_systems = json.load(file)

def generate_car_data(data_amount):
    cars = {
        'Car Model': [],
        'Autonomous Software': [],
        'Sensors': [],
        'Computing Units': [],
        'Connectivity': [],
        'Power System': [],
        'Braking System': [],
        'Steering System': [],
        'Price':[]
    }

    for _ in range(data_amount):
        # 随机选择品牌
        brand = random.choice(list(car_brands_models_systems.keys()))
        # 根据品牌随机选择型号
        model = random.choice(car_brands_models_systems[brand]['models'])
        car_model = f"{brand} {model['name']}"
        
        # 获取对应的自动驾驶系统和硬件配置
        software_system = car_brands_models_systems[brand]['autonomous_system']
        hardware = car_brands_models_systems[brand]['hardware']
        price=model['price']
        # 添加到各列
        cars['Car Model'].append(car_model)
        cars['Autonomous Software'].append(software_system)
        cars['Price'].append(price)
        for system, component in hardware.items():
            cars[system].append(component)

    return pd.DataFrame(cars)

def create_combined_dataframe(data_amount):
    # 创建 Faker 实例，指定英文环境
    fake = Faker('en_US')

    # 生成随机英文人名
    random_names = [fake.name() for _ in range(data_amount)]

    # 生成随机英文地址
    random_addresses = [fake.address() for _ in range(data_amount)]

    # 生成最近一年的过去日期
    past_dates = [fake.past_date(start_date='-50d') for _ in range(int(data_amount/2))]

    # 生成最近一年的未来日期
    future_dates = [fake.future_date(end_date='+50d') for _ in range(int(data_amount/2))]

    # 合并过去和未来的日期
    fake_dates = past_dates + future_dates

    # 将日期转换为 datetime.datetime 类型
    fake_dates = [datetime.combine(date, datetime.min.time()) for date in fake_dates]

    # 获取今天的日期
    today = datetime.today()

    # 添加 "Status" 列
    status_list = []
    for date in fake_dates:
        if date < today:
            status_list.append('Expired')
        elif 0 <= (date - today).days <= 7:
            status_list.append('Expiring in 7 Days')
        else:
            status_list.append('Active')

    # 生成在 $80 到 $160 之间均匀分布的月保费金额，并保留两位小数
    monthly_premiums = np.round(np.random.uniform(80, 160, data_amount), 2)

    # 创建初始 DataFrame
    df = pd.DataFrame({
        'Client': random_names,
        'Address': random_addresses,
        'Expiration Date': fake_dates,
        'Status': status_list,
        'Premium': monthly_premiums
    })

    # 提取地址中的州信息
    def extract_state(address):
        match = re.search(r'\b[A-Z]{2}\b', address)
        if match:
            return match.group(0)
        return None

    # 添加 "State" 列
    df['State'] = df['Address'].apply(extract_state)

    # 生成汽车数据
    car_data_df = generate_car_data(data_amount)

    # 合并 DataFrame
    df = pd.concat([df, car_data_df], axis=1)

    return df

# 生成 1000 条数据
data_amount = 1000
df = create_combined_dataframe(data_amount)
# 将 car_data 生成的 DataFrame 添加到现有 DataFrame 中
car_data_df = generate_car_data(data_amount)

# 合并 DataFrame
df = pd.concat([df, car_data_df], axis=1)



In [14]:
df

,Client,Address,Expiration Date,Status,Premium,State,Car Model,Autonomous Software,Sensors,Computing Units,...,Price,Car Model,Autonomous Software,Sensors,Computing Units,Connectivity,Power System,Braking System,Steering System,Price
0,Jennifer Moore,"42293 Blackburn Vista Apt. 229\nWalkerstad, MP...",2024-09-18,Expired,100.23,MP,Mercedes-Benz GLS,Mercedes-Benz Drive Pilot,LiDAR + Camera,NVIDIA Xavier,...,76000,BYD Song Plus,BYD DiPilot,LiDAR,NVIDIA Orin,Bluetooth Adapter,DC-DC Converter,ABS Module,Electric Power Steering,32000
1,Timothy Blake,"943 Jacob Fall Apt. 769\nJenningsfort, MO 69738",2024-09-11,Expired,145.15,MO,Mercedes-Benz EQS,Mercedes-Benz Drive Pilot,LiDAR + Camera,NVIDIA Xavier,...,102000,XPeng P8,XPilot,LiDAR,Xavier AI Chip,Wi-Fi Module,Battery Pack,Electronic Brake Booster,Steering Actuator,30000
2,Erica Graham,"7572 Thompson Manor\nNew Cody, MH 64370",2024-10-24,Expired,125.36,MH,Audi e-tron,Audi AI,Radar + LiDAR,NVIDIA Jetson,...,66000,Mercedes-Benz E-Class,Mercedes-Benz Drive Pilot,LiDAR + Camera,NVIDIA Xavier,5G Modem,Inverter,ABS Module,Electric Power Steering,54000
3,Stacy Daniel,"677 Flowers Point Suite 078\nLake Jessica, FM ...",2024-09-28,Expired,135.27,FM,BMW iX,BMW iDrive,Camera + Radar,NVIDIA Orin,...,83000,BYD Han,BYD DiPilot,LiDAR,NVIDIA Orin,Bluetooth Adapter,DC-DC Converter,ABS Module,Electric Power Steering,30000
4,Ashlee Summers,"42862 Jefferson Course\nSmithchester, NC 24383",2024-09-29,Expired,97.03,NC,Audi Q8 e-tron,Audi AI,Radar + LiDAR,NVIDIA Jetson,...,74000,Audi Q8 e-tron,Audi AI,Radar + LiDAR,NVIDIA Jetson,5G Modem,Battery Pack,ABS Module,Electric Power Steering,74000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Daniel Duncan,"7766 Ann Shoals\nDeckerberg, NY 99003",2024-11-15,Active,138.67,NY,Audi A6 e-tron,Audi AI,Radar + LiDAR,NVIDIA Jetson,...,70000,BMW i4,BMW iDrive,Camera + Radar,NVIDIA Orin,Wi-Fi Module,Battery Pack,Electronic Brake Booster,Steering Actuator,56000
996,Todd Jones,"202 Murray Parkway\nKaylatown, DC 70455",2024-12-08,Active,91.80,DC,Li Auto L9,Li AD Max,Camera + LiDAR,NVIDIA Orin,...,68000,Mercedes-Benz GLS,Mercedes-Benz Drive Pilot,LiDAR + Camera,NVIDIA Xavier,5G Modem,Inverter,ABS Module,Electric Power Steering,76000
997,Felicia Owens,"6858 Heather Hollow Apt. 060\nJonathanhaven, A...",2024-10-29,Expiring in 7 Days,110.53,AZ,Li Auto L7,Li AD Max,Camera + LiDAR,NVIDIA Orin,...,50000,BYD Tang,BYD DiPilot,LiDAR,NVIDIA Orin,Bluetooth Adapter,DC-DC Converter,ABS Module,Electric Power Steering,35000
998,Sheila Matthews,"771 Johnson Estate Suite 467\nWeberberg, AK 60429",2024-11-07,Active,96.01,AK,BYD Han,BYD DiPilot,LiDAR,NVIDIA Orin,...,30000,XPeng P7,XPilot,LiDAR,Xavier AI Chip,Wi-Fi Module,Battery Pack,Electronic Brake Booster,Steering Actuator,35000


In [8]:
import numpy as np
import pandas as pd
# 模拟过去36个月的数据
num_months = 36

# 初始化数据列表
months = []
policies_sold_list = []
total_premium_list = []
insured_amount_list = []
settled_claims_list = []
outstanding_claims_list = []
settled_loss_list = []
outstanding_loss_list = []
loss_ratio_list = []
expense_ratio_list = []
combined_ratio_list = []
net_profit_list = []
net_profit_margin_list = []

# 模拟过去36个月的数据
for i in range(num_months):
    # 时间戳
    month = pd.Timestamp.now() - pd.DateOffset(months=(num_months - i - 1))
    
    # 保单概览
    policies_sold = int(np.random.uniform(800, 1200))
    total_premium = np.round(np.random.uniform(1_500_000, 4_000_000), 2)
    insured_amount = np.round(total_premium * np.random.uniform(10, 15), 2)
    
    # 理赔概览
    settled_claims = int(np.random.uniform(200, 400))
    outstanding_claims = int(np.random.uniform(50, 100))
    
    # 保证理赔金额处于合理范围以保持盈利
    settled_loss = np.round(settled_claims * np.random.uniform(4_000, 10_000), 2)
    outstanding_loss = np.round(outstanding_claims * np.random.uniform(4_000, 10_000), 2)
    
    # 管理概览
    loss_ratio = np.round((settled_loss + outstanding_loss) / total_premium * 100, 2)
    
    # 调整费用率使利润率在目标范围
    target_expense_ratio = max(0, 100 - loss_ratio - 3)  # 目标平均利润率约为3%
    expense_ratio = np.round(np.random.uniform(target_expense_ratio - 5, target_expense_ratio), 2)
    combined_ratio = np.round(loss_ratio + expense_ratio, 2)
    
    # 计算净利润和净利润率
    net_profit = np.round(total_premium - (settled_loss + outstanding_loss) - (total_premium * expense_ratio / 100), 2)
    net_profit_margin = np.round((net_profit / total_premium) * 100, 2)
    
    # 存储数据
    months.append(month.strftime('%Y-%m'))
    policies_sold_list.append(policies_sold)
    total_premium_list.append(total_premium)
    insured_amount_list.append(insured_amount)
    settled_claims_list.append(settled_claims)
    outstanding_claims_list.append(outstanding_claims)
    settled_loss_list.append(settled_loss)
    outstanding_loss_list.append(outstanding_loss)
    loss_ratio_list.append(loss_ratio)
    expense_ratio_list.append(expense_ratio)
    combined_ratio_list.append(combined_ratio)
    net_profit_list.append(net_profit)
    net_profit_margin_list.append(net_profit_margin)

# 创建 DataFrame
df_insurance = pd.DataFrame({
    'Month': months,
    'Policies Sold': policies_sold_list,
    'Total Premium': total_premium_list,
    'Insured Amount': insured_amount_list,
    'Settled Claims': settled_claims_list,
    'Outstanding Claims': outstanding_claims_list,
    'Settled Loss': settled_loss_list,
    'Outstanding Loss': outstanding_loss_list,
    'Loss Ratio (%)': loss_ratio_list,
    'Expense Ratio (%)': expense_ratio_list,
    'Combined Ratio (%)': combined_ratio_list,
    'Net Profit': net_profit_list,
    'Net Profit Margin (%)': net_profit_margin_list
})

# 计算实际的平均净利润率
average_net_profit_margin = df_insurance['Net Profit Margin (%)'].mean()

# 显示结果
print(df_insurance.head())
print("\nAverage Net Profit Margin: {:.2f}%".format(average_net_profit_margin))

     Month  Policies Sold  Total Premium  Insured Amount  Settled Claims  \
0  2021-11           1094     1545144.16     21109104.73             309   
1  2021-12            843     1715430.28     20830259.43             268   
2  2022-01           1139     1549612.35     21287138.35             398   
3  2022-02           1050     2782903.29     37926642.54             250   
4  2022-03           1086     1643348.31     22932063.58             296   

   Outstanding Claims  Settled Loss  Outstanding Loss  Loss Ratio (%)  \
0                  59    2513406.39         539361.09          197.57   
1                  66    2079529.32         345135.32          141.34   
2                  61    3979251.33         519079.70          290.29   
3                  52    1301719.62         279708.86           56.83   
4                  55    2395906.02         533884.87          178.28   

   Expense Ratio (%)  Combined Ratio (%)  Net Profit  Net Profit Margin (%)  
0              -3.25      

In [9]:
df_insurance

,Month,Policies Sold,Total Premium,Insured Amount,Settled Claims,Outstanding Claims,Settled Loss,Outstanding Loss,Loss Ratio (%),Expense Ratio (%),Combined Ratio (%),Net Profit,Net Profit Margin (%)
0,2021-11,1094,1545144.16,21109104.73,309,59,2513406.39,539361.09,197.57,-3.25,194.32,-1457406.13,-94.32
1,2021-12,843,1715430.28,20830259.43,268,66,2079529.32,345135.32,141.34,-1.11,140.23,-690193.08,-40.23
2,2022-01,1139,1549612.35,21287138.35,398,61,3979251.33,519079.70,290.29,-0.85,289.44,-2935546.98,-189.44
3,2022-02,1050,2782903.29,37926642.54,250,52,1301719.62,279708.86,56.83,39.86,96.69,92209.56,3.31
4,2022-03,1086,1643348.31,22932063.58,296,55,2395906.02,533884.87,178.28,-2.79,175.49,-1240593.16,-75.49
5,2022-04,1106,2850768.54,31040366.64,261,51,1501281.41,507743.11,70.47,25.81,96.28,105960.66,3.72
6,2022-05,1098,1975720.80,22953106.18,263,66,2456869.57,617283.96,155.60,-2.18,153.42,-1055362.02,-53.42
7,2022-06,1180,1511572.03,21974589.45,247,84,1901923.21,437023.09,154.74,-3.78,150.96,-770236.85,-50.96
8,2022-07,825,3747042.38,45320549.65,267,72,1337962.86,384843.47,45.98,47.90,93.88,229402.75,6.12
9,2022-08,1048,2880676.81,30977827.93,215,61,1910183.42,279714.84,76.02,17.98,94.00,172832.86,6.00
